# Modelado en Optimización (IIND-2501)

## Lección 3.4: '*Overview*' del Método Simplex

**Propósito.**
- Emular una iteración de Simplex: base \(B\), no-base \(N\), multiplicadores, costos reducidos, **dirección** y **longitud de paso**.
- Conectar con la intuición de *moverse de vértice en vértice* intercambiando variables básicas/no básicas.

**Cómo usar.**
1. Edita la **celda de datos** para que coincida con el Taller 18.
2. Ejecuta el notebook. Usa las celdas de *cálculo de una iteración* para realizar un pivote (con selección manual de entrante/saliente).
3. Repite si quieres varias iteraciones.


# Pasos del método Simplex

1. Elegir una **base factible inicial**.

   $$x = \left[ \begin{array}{c} x_B \\ \hline x_N \end{array}\right] =
    \left[
    \begin{array}{c}
    B^{-1}b \\
    \hline
    0
    \end{array}
    \right], z=c_B^{T}x_B$$

2. **Verificar optimalidad**: calcular **costos reducidos** (recordar $w^{T} = c_B^{T} B^{-1} $)
   $$r = c_N - (c_B^{T} B^{-1}) N = c_N - w^{T} N$$
   
Si existe al menos un $r_q > 0$ para maximización (o un $r_q < 0$ para minimización), la variable no básica `q` **entra a la base** (pues su $r_q$ promete una mejora en la función objetivo).
   
3. **Definir dirección** (a partir de la variable entrante $q$, si la hay):

    $$d^{q} = \left[ \begin{array}{c} d^{q}_B \\ \hline d^{q}_N \end{array}\right] =
    \left[
    \begin{array}{c}
    -B^{-1} A_q \\
    \hline
    e_q
    \end{array}
    \right]$$
   
4. **Longitud**
ght]
\]

6. **Longitjd**:

   $$\theta = \min_{j\in ^{q}B}j{ \quad  \frac{-x_^{q}{ij}{d_{i}} \quad | \quad d_{i} < 0 \quad  \} $$

Si ninguna componente de la dirección es negativa, el problema es **no acotado**.

8. **Pivoteo**: actualizar base (sale la variable básica) y repetir.

A continuación, un *widget* que deja **correr los pasos manualmente** y ver la geometría en 2D.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Math, HTML
import ipywidgets as W
from ipywidgets import HTMLMath

from pathlib import Path
import sys
sys.path.append(str(Path.cwd() / "src"))

%load_ext autoreload
%autoreload 2

from helpers_simplex import (
    mat_to_bmatrix, vec_to_bmatrix,
    feasible_polygon, plot_method_graph,
    basis_info, step_info
)

In [2]:
def _blk(s):
    # Negro: info de la base actual
    return f"<div style='color:#111; font-weight:500;'>{s}</div>"

def _blu(s):
    # Azul: x y costos reducidos
    return f"<div style='color:#1e3a8a; font-weight:500;'>{s}</div>"

def _red(s):
    # Rojo: movimiento (dirección/longitud/nuevo punto)
    return f"<div style='color:#b91c1c; font-weight:600;'>{s}</div>"
'''
def _append_base(html_chunk):   # negro + azul
    log_base.value += html_chunk

def _append_move(html_chunk):   # rojo
    log_move.value += html_chunk
'''
log_base = HTMLMath(value="")   # negro + azul
log_move = HTMLMath(value="")   # rojo

In [3]:
# =============================================================
# 1) PROBLEMA (mismo del notebook/imagen: NO cambiar datos)
# =============================================================
# max 3 x1 + 2 x2
# s.a.
#   x1 +  x2 <= 6
#   x1 + 2x2 <= 8
#   2x1 + x2 <= 8
#   x1, x2 >= 0
# Forma estándar con 3 slacks: [x1, x2, s1, s2, s3]

A = np.array([
    [1., 1., 1., 0., 0.],
    [1., 2., 0., 1., 0.],
    [2., 1., 0., 0., 1.],
])
b = np.array([6., 8., 8.])
c = np.array([3., 2., 0., 0., 0.])

# Base inicial: slacks {3,4,5}, no básicos {1,2}
Bcols = [0,0,1,1,1]  # índices 0-based (2,3,4 == s1,s2,s3)

In [4]:
# Figura única: región factible + BFS
fig, ax = plt.subplots(figsize=(6,5))
ax, line_colors, hull = plot_method_graph(A[:, :2], b, x=None, ax=ax,
                                          title="Método gráfico + BFS", show_fill=True)
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
plt.close(fig)

fig_widget = W.Output()
with fig_widget:
    display(fig)

# Estado mutable del dashboard
STATE = {
    "A": A, "b": b, "c": c,
    "Bcols": Bcols.copy(),
    "info": None,          # dict de basis_info
    "last_enter": None,    # índice de variable entrante (0-based)
    "x_plot": None,        # punto actual en R^2
}

In [5]:
# -------------------------------------------------------------
#    Funciones PSEUDO-ALGORITMO SIMPLEX  (devuelven bloque HTML para Markdown)
# -------------------------------------------------------------

def resumir_info_base_actual(A, b, c, Bcols):
    """Calcula B,N,cB,cN,x (básico) y devuelve (info_dict, html_negro).
    NO modifica el estilo ni datos. Solo resume la base actual.
    """
    
    info = basis_info(A, b, c, Bcols)
    B, N = info["B"], info["N"]
    x, xB = info["x"], info["xB"]
    cB, cN = info["cB"], info["cN"]
    I_B = [j+1 for j,v in enumerate(Bcols) if v]  # 1-based para mostrar
    I_N = [j+1 for j in info["Ncols"]]

    # Punto actual en R^2
    x2d = x[:2]   
    html_blk = fr"""
        <h4 style="color:#111">Base actual: $I_B = {I_B}$, $I_N = {I_N}$</h4>        
        
        <p>$$c_B^T = {mat_to_bmatrix(cB.reshape(1,-1))},\quad
        c_N^T = {mat_to_bmatrix(cN.reshape(1,-1))}$$</p>
        
        <p>$$B = {mat_to_bmatrix(B)},\quad
        N = {mat_to_bmatrix(N)},\quad
        b = {vec_to_bmatrix(b)}$$</p>                        
        """
        #$$x_B = B^{{-1}} b = {vec_to_bmatrix(xB)};\;\; x = {vec_to_bmatrix(x[:2])}$$
    #log_base.value += html_blk;
    return info, html_blk


def verificar_costos_reducidos(info):
    """Con info de la base calcula y,z,r y devuelve bloque HTML azul."""
    y = info["y"]
    r = info["r"]
    x2d = info["x"][:2]
    B_idx = [i for i in range(len(info["Bcols"])) if info["Bcols"][i]]
    z = float((info["cB"]).transpose() @ info["x"][B_idx])  # z = c_B^T x_B

    html_blue = fr"""      
        <p><b>Solución básica actual:</b>
        <p>$$x = {vec_to_bmatrix(info["x"])},\;\; z = {z:.2f},\;\; w^{{T}} = c_B^{{T}} B^{{-1}} = {vec_to_bmatrix(y)}$$</p>        
        <p>Costos reducidos:
        $$r = c_N - w^{{T}} N = {vec_to_bmatrix(info["r"])}$$
        </p>
        """
    #log_base.value += html_blue;
    return html_blue


def calcular_direccion_y_longitud(A, info, enter_j):
    """
    A partir de la base e índice entrante (0-based), calcula dirección d,
    longitud θ (regla del mínimo cociente), variable saliente y x nuevo.
    Devuelve (step, html_rojo).
    """
    st = step_info(A, info["x"], info["Bcols"], enter_j)
    
    if np.isfinite(st["theta"]):
        html_red = fr"""
        <h4 style="color:#b91c1c">Movimiento:</h4>        
        <p>Entra: {var_names[STATE['last_enter']]}; esto induce $d^{STATE['last_enter']+1}= -B^{{-1}}a_{STATE['last_enter']+1}$</p>
        <p>$$x^{{t+1}} = x^{{t}} + \alpha d^{STATE['last_enter']+1} $$</p>
        <p>$${vec_to_bmatrix(st["x_new"])} = {vec_to_bmatrix(info["x"])} + {st['theta']}{vec_to_bmatrix(st["d"])}$$</p>
        <p>(sale variable: {st['leave']+1}; i.e., {var_names[st['leave']]})</p>
        """
        #<p>Dirección $d$ = {vec_to_bmatrix(st["d"])}</p>
        #<p>Longitud $\theta$ = {st["theta"]:.4g} \; \Rightarrow; sale var {st["leave"]+1}</p>
    else:
        html_red = fr"""
        <h4 style="color:#b91c1c">Movimiento</h4>
        <p>Problema no acotado en la dirección escogida (no hay $\theta$ finito).</p>
        """
    #log_move.value += html_red;
    return st, html_red

In [6]:
# =============================================================
# CALLBACKS DE LOS BOTONES
# =============================================================

def _draw_point_and_vectors(x_now=None, d=None, theta=None):
    # Limpia y repinta: región factible + líneas (se conserva estilo)
    ax.clear()
    plot_method_graph(A[:, :2], b, x=None, ax=ax, title="Método gráfico + BFS", show_fill=True)
    # Punto actual en negro
    if x_now is not None:
        ax.scatter([x_now[0]], [x_now[1]], s=60, color="black", zorder=6)
    # Flecha de dirección y nuevo punto en rojo
    if (d is not None) and (theta is not None) and np.isfinite(theta):
        p = x_now
        q = x_now + theta * d[:2]
        ax.annotate("", xy=(q[0], q[1]), xytext=(p[0], p[1]),
                    arrowprops=dict(arrowstyle="->", linewidth=2, color="red"))
        ax.scatter([q[0]], [q[1]], s=60, color="red", zorder=6)
    fig_widget.clear_output(wait=True)
    with fig_widget:
        display(fig)


# === CALLBACKS ===
def _on_info_clicked(_):
    log_base.value = ""     
    log_move.value = ""  

    try:
        # === Leer checkboxes y fijar Bcols ===
        seleccionados = [chk.value for j, chk in enumerate(checks.children, start=1)]
        STATE["Bcols"] = seleccionados

        # === Resto del flujo ===
        info, html_blk = resumir_info_base_actual(STATE["A"], STATE["b"], STATE["c"], STATE["Bcols"])
        STATE["info"] = info

        var_names = ['x1','x2','s1','s2','s3']
        nonbasic = [(v,i) for i,v in enumerate(var_names) if not STATE["Bcols"][i]]
        enter_drop.options = nonbasic
        enter_drop.value = nonbasic[0][1]

        log_base.value += _blk(html_blk)
        html_blue = verificar_costos_reducidos(info)
        log_base.value += _blu(html_blue)

        STATE["x_plot"] = info["x"][:2]
        _draw_point_and_vectors(x_now=STATE["x_plot"])

    except Exception as e:
        log_base.value += f"<pre style='color:#b91c1c'>[error info] {e}</pre>"


def _on_move_clicked(_):
    log_move.value = ""     # opcional: limpiar solo el panel rojo
    try:
        if STATE["info"] is None:
            log_move.value += "<div style='color:#b91c1c'>Primero pulsa “1) Info Base Actual”.</div>"
            return
        j_in = enter_drop.value
        STATE["last_enter"] = j_in

        st, html_red = calcular_direccion_y_longitud(STATE["A"], STATE["info"], j_in)        
        log_move.value += _red(html_red)

        if np.isfinite(st["theta"]):
            _draw_point_and_vectors(x_now=STATE["x_plot"], d=st["d"], theta=st["theta"])
            STATE["x_plot"] = st["x_new"][:2]
            if st["leave"] is not None:
                '''
                B_idx = [i for i,v in enumerate(STATE["Bcols"]) if v]
                print(B_idx)
                Bset = set(B_idx) - {st["leave"]}
                Bset.add(j_in)
                aux = [(i in Bset) for i in range(STATE["A"].shape[1]) ]
                print(aux)
                '''                
                STATE["Bcols"][j_in] = True
                STATE["Bcols"][st["leave"]] = False
    except Exception as e:
        log_move.value += f"<pre style='color:#b91c1c'>[error move] {e}</pre>"

In [7]:
# === LAYOUT COMPACTO ===
checks = W.HBox([
    W.Checkbox(value=False, description='x1'),
    W.Checkbox(value=False, description='x2'),
    W.Checkbox(value=True, description='s1'),
    W.Checkbox(value=True, description='s2'),
    W.Checkbox(value=True, description='s3'),
])
btn_info = W.Button(description="1) Info Base", button_style="primary")

var_names = ['x1','x2','s1','s2','s3']
nonbasic = [(v,i) for i,v in enumerate(var_names) if not STATE["Bcols"][i]]
enter_drop = W.Dropdown(description="Entra:", options=nonbasic, value=nonbasic[0][1])
btn_move = W.Button(description="2) Mover", button_style="warning")

controls = W.HBox([btn_info, enter_drop, btn_move])
right_panel = W.VBox([log_base, W.HTML("<hr>"), log_move])

ui = W.HBox([
    W.VBox([checks, fig_widget], layout=W.Layout(width='60%')),
    W.VBox([controls, right_panel], layout=W.Layout(width='40%'))
])

# Registro idempotente de callbacks (clave para quitar duplicados)
btn_info.on_click(_on_info_clicked, remove=True)
btn_info.on_click(_on_info_clicked)

btn_move.on_click(_on_move_clicked, remove=True)
btn_move.on_click(_on_move_clicked)

display(ui)

# (opcional) Dibuja la región inicialmente
_draw_point_and_vectors()